In [1]:
!pip install scikit-learn==1.3.2 numpy==1.24.3 --force-reinstall

  Using cached scikit_learn-1.3.2-cp311-cp311-macosx_12_0_arm64.whl (9.4 MB)
  Using cached numpy-1.24.3-cp311-cp311-macosx_11_0_arm64.whl (13.8 MB)
  Using cached scipy-1.13.1-cp311-cp311-macosx_12_0_arm64.whl (30.3 MB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.5.0
    Uninstalling threadpoolctl-3.5.0:
      Successfully uninstalled threadpoolctl-3.5.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: scikit-

In [2]:
import sklearn; print(sklearn.__version__)

1.3.2


In [3]:
# Necessary imports 
import numpy as np 
import pandas as pd 
from xgboost import XGBRegressor 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.preprocessing import OneHotEncoder

In [4]:
# Load the data 
dataset = pd.read_csv("validator_data.csv") 
X = dataset.drop(['effectiveness'], axis=1)
y = dataset['effectiveness']


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5388 entries, 0 to 5387
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      5388 non-null   object 
 1   total_validators        5388 non-null   int64  
 2   p_active_slots          5388 non-null   int64  
 3   p_created_blocks        5388 non-null   int64  
 4   non_empty_block         5329 non-null   float64
 5   empty_block             5329 non-null   float64
 6   vanilla_blocks          5329 non-null   float64
 7   a_active_epochs         5388 non-null   int64  
 8   a_attested_epochs       5388 non-null   int64  
 9   a_attested_source       5388 non-null   int64  
 10  a_attested_target       5388 non-null   float64
 11  a_attested_head         5388 non-null   int64  
 12  inc_distance            5388 non-null   float64
 13  sync_rate               5165 non-null   float64
 14  block_reward            5388 non-null   

In [6]:
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X.select_dtypes(include= ['object'])))
# One-hot encoding removed index; put it back
OH_cols_train.index = X.index

In [7]:
OH_cols_train.head()

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X.select_dtypes(include= ['int64','float64'])
#  Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)

In [9]:
y.head()

0    0.977869
1    0.968877
2    0.969973
3    0.984954
4    0.985743
Name: effectiveness, dtype: float64

In [10]:
# Splitting 
train_X, test_X, train_y, test_y = train_test_split(OH_X_train, y, 
					test_size = 0.3, random_state = 123) 

In [11]:
# Instantiation 
xgb_r = XGBRegressor(objective ='reg:linear', n_estimators = 10, seed = 123) 


In [12]:
# Fitting the model 
xgb_r.fit(train_X, train_y) 


/Users/ashu/.pyenv/versions/3.11.0/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:01:01] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, objective='reg:linear', ...)

In [16]:

# Predict the model 
pred = xgb_r.predict(test_X) 
pred, test_X


(array([0.98389465, 0.98052675, 0.9786934 , ..., 0.95115924, 0.9864796 ,
        0.9834012 ], dtype=float32),
       total_validators  p_active_slots  p_created_blocks  non_empty_block  \
 1266              8906              61                61             61.0   
 3167              8852              54                54             54.0   
 1983              8991              94                93             93.0   
 4835              8697              85                85             85.0   
 1972               500               6                 6              6.0   
 ...                ...             ...               ...              ...   
 3006              8490              64                63             63.0   
 142              10000              80                79             79.0   
 3405              8750              64                64             64.0   
 1727              8980              61                61             61.0   
 1995              9744         

In [14]:
# RMSE Computation 
rmse = np.sqrt(MSE(test_y, pred)) 
print("RMSE : % f" %(rmse)) 

RMSE :  0.002624
